In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules import Module
import random
from torch.autograd import Variable
import decimal
import os

In [ ]:
# hyperparameters
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 90

In [ ]:
# datafn
def devision_data(size):
    xdata = []
    ydata = []
    for i in range(int(size/BATCH_SIZE)):
        xbatch = []
        ybatch = []
        for j in range(BATCH_SIZE):
            i1, i2 = float(decimal.Decimal(random.randrange(100, 2000))/100), float(decimal.Decimal(random.randrange(100, 2000))/100)
            y = i1 / i2 / 100
            xbatch.append([i1, i2])
            ybatch.append([y])
        xbatch = torch.tensor(xbatch, dtype=torch.float)
        ybatch = torch.tensor(ybatch)
        xdata.append(xbatch)
        ydata.append(ybatch)
    return list(zip(xdata, ydata))

train_data = devision_data(64000)
test_data = devision_data(12800)

In [ ]:
# custom activation function
class CustomActivation(Module):
    def forward(self, x):
        x = torch.where(
            x <= 0, 1.359140915 * (x-1).exp(), 
            torch.where(
                x > 15, 1 - 1/(109.0858178 * x - 1403.359435), 
                0.03 * (1000000 * x + 1).log() + 0.5
            )
        )
        return x

In [ ]:
# model
act_fn = CustomActivation()
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = nn.Linear(2, 2)
        self.l2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.l1(x)
        x = act_fn(x)
        x = self.l2(x)
        x = act_fn(x)
        return x

In [ ]:
# laod
load = input('load? y/n ')
if load == 'y':
    model = torch.load('./model/Torch.pth')
    while True:
        inputs = input('\ninputs: ')
        try:
            print(model(torch.tensor([int(inputs.split(',')[0].strip()), int(inputs.split(',')[1].strip())], dtype=torch.float)).item() * 20)
        except:
            exit()
else:
    model = Model()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

def train(epoch):
    model.train()
    for batch_id, (data, target) in enumerate(train_data):
        data = Variable(data)
        target = Variable(target)
        Y_pred = model(data)
        loss = criterion(Y_pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch + 1, batch_id * len(data), BATCH_SIZE * len(train_data),
                    100. * batch_id / len(train_data), loss.item()))

def test():
    model.eval()
    totalLoss = 0
    for data, target in test_data:
        data = Variable(data)
        target = Variable(target)
        Y_pred = model(data)
        loss = criterion(Y_pred, target)
        totalLoss += loss
    print('Custom Durchschnittsloss: ', totalLoss / len(test_data))

# for epoch in range(EPOCHS):
#     train(epoch)
#     test()

# 3.8294e-15
print(model.l1.weight, model.l1.bias, model.l1.weight.shape, model.l1.bias.shape)
print(model.l2.weight, model.l2.bias, model.l2.weight.shape, model.l2.bias.shape)

In [ ]:
l1HardWeights = torch.tensor([[0.000001, 0.0], [0.0, 0.000001]])
l2HardWeights = torch.tensor([[33.3333, -33.3333]])
l1HardBias = torch.tensor([-0.000001, -0.000001])
l2HardBias = torch.tensor([-3.912023])
model.l1.weight = torch.nn.Parameter(l1HardWeights)
model.l2.weight = torch.nn.Parameter(l2HardWeights)
model.l1.bias = torch.nn.Parameter(l1HardBias)
model.l2.bias = torch.nn.Parameter(l2HardBias)
# print(model.l1.weight, model.l1.bias, model.l1.weight.shape, model.l1.bias.shape)
# print(model.l2.weight, model.l2.bias, model.l2.weight.shape, model.l2.bias.shape)

test()

In [ ]:
# save
save = input('save? y/n ')
if save == 'y':
    model_folder_path = './model'
    file_name='Torch.pth'
    if not os.path.exists(model_folder_path):
        os.makedirs(model_folder_path)

    file_name = os.path.join(model_folder_path, file_name)
    torch.save(model, file_name)